This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [3]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

sYnhA1u2Yxaq-ptMCJsj


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [14]:
# First, import the relevant modules

import requests, json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

In [149]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned


### Json request
url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}"

json_data_req = requests.get(url)

# 2. Convert the returned JSON object into a Python dictionary.

In [148]:
json_data_dict = json.loads(json_data_req.content)

# setting up list for just stock prices by date

In [100]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


#print(data)
stock_data_list = [i for i in json_data_dict['dataset_data']['data']]
#print(stock_data_array)
stock_data_list = sorted(stock_data_list, key=lambda x: x[0]) # sorting by date 


[('Date', 0), ('Open', 1), ('High', 2), ('Low', 3), ('Close', 4), ('Change', 5), ('Traded Volume', 6), ('Turnover', 7), ('Last Price of the Day', 8), ('Daily Traded Units', 9), ('Daily Turnover', 10)]


In [116]:
# check index of column names of stock_data_list 
column_names = json_data_dict['dataset_data']['column_names']
print(list(zip(column_names, range(len(column_names)))))

[('Date', 0), ('Open', 1), ('High', 2), ('Low', 3), ('Close', 4), ('Change', 5), ('Traded Volume', 6), ('Turnover', 7), ('Last Price of the Day', 8), ('Daily Traded Units', 9), ('Daily Turnover', 10)]


# 3. Calculate what the highest and lowest opening prices were for the stock in this period. 

In [88]:
# finding the open

open_prices_list = [i[1] for i in stock_data_list if i[1] != None] # removing all None values for open prices
min_open_price = min(open_prices_list)
max_open_price = max(open_prices_list)

print(f'min open price = {min_open_price} \nmax open price = {max_open_price}')


min open price = 34.0 
max open price = 53.11


# 4. What was the largest change in any one day (based on High and Low price)?

In [114]:
high_low_price_list = [[i[0:4], i[2]-i[3]]for i in stock_data_list ]
change_list = [i[1] for i in high_low_price_list]
max_high_low = max(change_list)
date_biggest_change = [i for i in high_low_price_list if i[1] == max_high_low] # more than 1 if multiple dates have the same change

print(f'Largest Change in any one day: {max_high_low} on {date_biggest_change[0][0][0]}')

Largest Change in any one day: 2.8100000000000023 on 2017-05-11


# 5. What was the largest change between any two days (based on Closing Price)?

In [130]:
# stock_data_list closing price 4  .
# calculate on each day -->  2 days before close - current day close
# largest change positive , largest change negative and overall largest change
# problem: first 2 days don't have 2 days prior data (ignore this issue for this project NA for those first 2 days)
# problem: is it two actual days? or 2 trading days. for this project, I will use trading days

two_day_change_list = []
for i in range(len(stock_data_list)):
    current_date = stock_data_list[i][0]
    current_date_close = stock_data_list[i][4]
    if i-2 >= 0:
        two_days_ago_close = stock_data_list[i-2][4]
    else:
        two_days_ago_close = current_date_close
    two_day_change = two_days_ago_close - current_date_close
        
    single_date_data = [current_date, two_days_ago_close, current_date_close, two_day_change ]
    two_day_change_list.append(single_date_data)

max_positive_two_day_change = max([i[3] for i in two_day_change_list])
min_positive_two_day_change = min([i[3] for i in two_day_change_list])

print (f'Largest Positive two day change: {max_positive_two_day_change}\nLargest Negative two day change: {min_positive_two_day_change}')
  

Largest Positive two day change: 3.1499999999999986
Largest Negative two day change: -2.280000000000001


# 6. What was the average daily trading volume during this year?

In [146]:
trading_volume_list = [i[6] for i in stock_data_list]
avg_trading_volume = sum(trading_volume_list)/len(trading_volume_list)
print( f"Average daily trading volume: {avg_trading_volume}")

[44700.0,
 70618.0,
 54408.0,
 48412.0,
 27507.0,
 62225.0,
 43976.0,
 123530.0,
 163860.0,
 59367.0,
 47879.0,
 39195.0,
 65931.0,
 73105.0,
 80246.0,
 55333.0,
 48797.0,
 56240.0,
 64103.0,
 69657.0,
 69603.0,
 63371.0,
 85137.0,
 54279.0,
 40705.0,
 41911.0,
 67410.0,
 39731.0,
 67781.0,
 115843.0,
 177171.0,
 82601.0,
 77420.0,
 84682.0,
 60583.0,
 37014.0,
 46070.0,
 77619.0,
 81945.0,
 101294.0,
 62655.0,
 67440.0,
 86753.0,
 49863.0,
 40800.0,
 46510.0,
 61925.0,
 45111.0,
 43988.0,
 38518.0,
 44803.0,
 60420.0,
 60324.0,
 55799.0,
 90109.0,
 97572.0,
 56906.0,
 670349.0,
 169971.0,
 210926.0,
 191515.0,
 115075.0,
 335406.0,
 189201.0,
 95382.0,
 127973.0,
 193156.0,
 156005.0,
 96794.0,
 81255.0,
 62297.0,
 107817.0,
 158278.0,
 88416.0,
 88416.0,
 88416.0,
 133057.0,
 92722.0,
 223621.0,
 186784.0,
 102084.0,
 85973.0,
 65847.0,
 151683.0,
 86348.0,
 86348.0,
 86559.0,
 65266.0,
 82058.0,
 78512.0,
 97456.0,
 151439.0,
 91858.0,
 189125.0,
 159495.0,
 193702.0,
 101476.0,
 14

Average daily trading volume: 89124.33725490196


# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [147]:
sorted_trading_volume_list = sorted(trading_volume_list)
median_trading_volume = sorted_trading_volume_list[int(len(sorted_trading_volume_list)/2)]
print(f"Median Trading Volume: {median_trading_volume}")

Median Trading Volume: 76286.0


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)